In [ ]:

import pandas as pd
import os,read

In [ ]:

from langchain.agents import create_csv_agent
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.agents.agent_types import AgentType

In [ ]:

import importlib,utils,json,os
from utils import *
os.environ['OPENAI_API_KEY'] = read.read_json(os.path.join('configuration.json'))['OpenAI_api_key']



In [ ]:
read.read_json(os.path.join('configuration.json'))['OpenAI_api_key']


In [ ]:
gpt_eval=utils.read_json("gpt_eval_3000.json")

In [ ]:
gpt_eval[1].keys()

In [ ]:
import csv

def list_to_csv(data, file_path):
    with open(file_path, 'w', newline='') as csvfile:
        csv_writer = csv.writer(csvfile)
        csv_writer.writerows(data)

def list_to_df(data,):
    processed_data = [row[1:] for row in data[1:]]
    df = pd.DataFrame(processed_data)
    new_header = df.iloc[0] #grab the first row for the header
    df = df[1:] #take the data less the header row
    df.columns = new_header #set the header row as the df header
    return df


In [ ]:
# file_path = 'output_agent.csv'
# Pass a query to the data
for i,item in enumerate(gpt_eval[:1000]):
    if "LC_agent_res" not in item.keys():
        print(i)
        data=item["Table"]

        df=list_to_df(data)
        
        print(type(df) == pd.DataFrame)
        agent = create_csv_agent(
        ChatOpenAI(temperature=0.3, model="gpt-3.5-turbo"),
        path=None,
        pddf=df,
        verbose=True,
        agent_type=AgentType.OPENAI_FUNCTIONS
        )
        num_tries = 5
        for try_ in range(0,num_tries):
            try:
                a=agent.run(item["Question"])
            except Exception as e:
                print(e)
                print("failed, but we can try %d more time(s)" % (num_tries - try_ - 1))
                if try_ == num_tries-2:
                    a = 1
            else:
                #print("YESS!!! Success...")
                break
        # try:
            
        # except Exception as e:
        #     print(e)
        item["LC_agent_res"]=a
        ans=item ["gdres"]
        print(a,ans)

In [ ]:
with open('gpt_eval_agents.json', 'r') as f:
    gpt_eval=json.load(f)


In [ ]:
len(gpt_eval),gpt_eval[900].keys()

In [ ]:
for item in gpt_eval:
    if "LC_agent_res" in item and item["LC_agent_res"].startswith("It seems that there is"):
        print(item["LC_agent_res"])
        del item["LC_agent_res"]
        #print("ok")


In [ ]:
%store gpt_eval

In [ ]:
%store -r gpt_eval

In [ ]:
with open ("gpt_eval_agents_res.json","w") as f:
    json.dump(gpt_eval,f)

In [ ]:
a

In [ ]:
gpt_eval[0]["Question"]

In [ ]:
agent = create_csv_agent(
    OpenAI(temperature=0.7),
    file_path ,
    verbose=True,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
)
b=agent.run("Which team has an unknown coach and location of Athelstone?")

In [ ]:
loader = CSVLoader(file_path="./example_data/mlb_teams_2012.csv", source_column="Team")

data = loader.load()

In [ ]:
agent = create_csv_agent(
    OpenAI(temperature=0.7),
    "titanic.csv",
    verbose=True,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
)

In [ ]:

agent.run("how many rows are there?")


In [ ]:
loader = CSVLoader(file_path="./example_data/mlb_teams_2012.csv", source_column="Team")

data = loader.load()

In [ ]:
loader.load()

In [ ]:

def get_formular(question: str,table, model='gpt-3.5-turbo'):
    llm = ChatOpenAI(model_name=model, max_tokens=50,)
    prompt = "Write a excel formula to answer the quesion based on table beleow\n"\
        "You can use functions like SUM,XLOOKUP,UNIQUE,AVERAGE,etc...\n"\
             "{question}\n" \
             "Here is the table: {table}.\n" \
             "{format_instructions}"
             
    prompt_template = ChatPromptTemplate.from_template(prompt)
    
    #options = ResponseSchema(name="words", description="The words are related to bad behavior.")
    options = ResponseSchema(name="Formula", description="The formula is related to question")
    response_schemas = [options]
    output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
    form_instructions = output_parser.get_format_instructions()
    
    final_prompt = prompt_template.format_messages(question=question, table=table, format_instructions=form_instructions)
    print(final_prompt)
    output = llm(final_prompt)
    return output_parser.parse(output.content)



In [ ]:
len(gpt_eval)

In [ ]:
import csv

def list_to_csv(data, file_path):
    with open(file_path, 'w', newline='') as csvfile:
        csv_writer = csv.writer(csvfile)
        csv_writer.writerows(data)


file_path = 'output.csv'
# Pass a query to the data
for i,item in enumerate(gpt_eval):
    if "LC_res" not in item.keys():
        print(i)
        data=item["Table"]

        list_to_csv(data, file_path)
        #print("CSV file successfully created.")

        loader = CSVLoader(file_path=file_path)  

        # Create an index using the loaded documents
        index_creator = VectorstoreIndexCreator()
        docsearch = index_creator.from_loaders([loader])
        # Create a question-answering chain using the index
        chain = RetrievalQA.from_chain_type(llm=OpenAI(), chain_type="stuff", retriever=docsearch.vectorstore.as_retriever(), input_key="question")
        query=item ["Question"]
        print(query)
        response = chain({"question": query})
        item["LC_res"]=response['result']
        ans=item ["gdres"]
        print(response['result'],ans)

In [ ]:
for item in gpt_eval[:1]:
    a=str(item ["gdres"])
    b=str(item["LC_res"]["result"])
    if(a.lower()!=b.lower()):
    #if(1):
        print(a,b)
    else:
        print("\n\n")

In [ ]:
from thefuzz import fuzz
from thefuzz import process

In [ ]:
def fuzzy_cmp_str(a:str,b:str):
    a,b=str(a).lower(),str(b).lower()
    return fuzz.partial_ratio(a, b)

In [ ]:
thres=95
count=0
for i in range(len(gpt_eval)):
    radio=fuzzy_cmp_str(gpt_eval[i]["LC_res"],gpt_eval[i]["gdres"])
    #print(radio)
    if(radio>thres):
        count+=1
        print(gpt_eval[i]["LC_res"],"||",gpt_eval[i]["gdres"])
print(count/len(gpt_eval))